In [1]:
import pandas as pd
import numpy as np
#coding=utf-8
import numpy as np
import pandas as pd
from scipy.stats import mode
import xgboost as xgb
from xgboost import XGBRegressor
from xgboost import XGBClassifier
from sklearn.metrics import mean_squared_error,roc_auc_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from matplotlib import  pyplot
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor,RandomForestClassifier
from sklearn import preprocessing  
min_max_scaler = preprocessing.MinMaxScaler()  
from sklearn.model_selection import KFold,StratifiedKFold

In [2]:
train = pd.read_csv("../dealinput/train_bankcard.csv")
test = pd.read_csv("../dealinput/test_bankcard.csv")
train.columns
# train.rename(columns = {'trainAppl_sbm_tm':'Appl_sbm_tm'},inplace = True)   
# test.rename(columns = {'testAppl_sbm_tm':'Appl_sbm_tm'},inplace = True)  
# train.to_csv("../dealinput/train1.csv",index = False)
# test.to_csv("../dealinput/test1.csv",index = False)
fulldata = pd.concat([train,test],axis = 0,ignore_index = True)
train.head(),train.shape,test.shape
print(fulldata.isnull().sum().sort_values(ascending = False))
fulldata

target                  47031
card_type_more_count        0
bank_name_less_count        0
bank_name_more_cate         0
bank_name_more_count        0
card_type_less_cate         0
card_type_less_count        0
card_type_more_cate         0
havaCreditCard              0
phone_num                   0
id                          0
phoneSame                   0
phone_less_cate             0
phone_less_count            0
phone_more_cate             0
phone_more_count            0
bank_name_less_cate         0
dtype: int64


,bank_name_less_cate,bank_name_less_count,bank_name_more_cate,bank_name_more_count,card_type_less_cate,card_type_less_count,card_type_more_cate,card_type_more_count,havaCreditCard,id,phoneSame,phone_less_cate,phone_less_count,phone_more_cate,phone_more_count,phone_num,target
0,a,1,a,1,1.0,1,1.0,1,0,501951980776722440,0,132****2008,1,132****2008,1,1,0.0
1,c,1,c,1,1.0,1,1.0,1,0,525890212484616200,0,187****2500,1,187****2500,1,1,0.0
2,l,1,h,1,1.0,2,2.0,4,1,599309364691472392,0,135****3522,6,135****3522,6,6,0.0
3,m,1,m,1,1.0,1,1.0,1,0,582559913071350024,0,136****8651,1,136****8651,1,1,0.0
4,e,1,e,1,1.0,1,1.0,1,0,599017959926534408,0,137****4241,1,137****4241,1,1,0.0
5,b,1,a,4,1.0,2,2.0,4,1,575388917721731080,0,150****1035,6,150****1035,6,6,0.0
6,c,1,c,1,1.0,1,1.0,1,0,541887664329068552,0,134****2594,1,134****2594,1,1,0.0
7,c,1,c,1,1.0,1,1.0,1,0,605760099452981512,0,158****5755,1,158****5755,1,1,0.0
8,a,1,a,1,1.0,1,1.0,1,0,472286860756914440,0,188****8517,1,188****8517,1,1,0.0
9,i,1,o,1,1.0,3,1.0,3,0,600450091387457800,0,183****2222,3,183****2222,3,3,0.0


In [7]:
# 4.22 从这里开始，先将手机号和基础表中的手机号匹配是否相同（常见手机号）
# 本表就不要手机号的信息了
kk = pd.read_csv("../dealinput/train1.csv")   # train_auth_info
cc = pd.read_csv("../dealinput/test1.csv")   # train_auth_info
zz = pd.concat([kk,cc],axis = 0,ignore_index = True)
mm = zz[["id","phoneTop3"]]

fulldata = fulldata.merge(mm,how = "inner",on = "id")
fulldata.isnull().sum(),fulldata
# zz.columns
# # kk = pd.read_csv("../input/train_auth_info.csv")  
# print(kk.isnull().sum())
# # fulldata.info()


(bank_name_less_cate         0
 bank_name_less_count        0
 bank_name_more_cate         0
 bank_name_more_count        0
 card_type_less_cate         0
 card_type_less_count        0
 card_type_more_cate         0
 card_type_more_count        0
 havaCreditCard              0
 id                          0
 phoneSame                   0
 phone_less_cate             0
 phone_less_count            0
 phone_more_cate             0
 phone_more_count            0
 phone_num                   0
 target                  47031
 phoneTop3                   0
 dtype: int64,
        bank_name_less_cate  bank_name_less_count bank_name_more_cate  \
 0                        a                     1                   a   
 1                        c                     1                   c   
 2                        l                     1                   h   
 3                        m                     1                   m   
 4                        e                     1             

In [17]:
fulldata.phone_more_cate = fulldata.phone_more_cate.astype("str")
fulldata["phone_more_cate_top3"] = fulldata.phone_more_cate.apply(lambda x:x[0:3])
fulldata["phone_more_cate_top3"] = fulldata["phone_more_cate_top3"].astype("int64")
fulldata["auth_bank_phoneTop3_same"] = 0
for i in range(fulldata.shape[0]):
    if fulldata["phone_more_cate_top3"].iloc[i] != fulldata["phoneTop3"].iloc[i]:
        fulldata["auth_bank_phoneTop3_same"].iloc[i] = 1
fulldata

d:\gongju\python3.6 64\anzhuang\lib\site-packages\pandas\core\indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


,bank_name_less_cate,bank_name_less_count,bank_name_more_cate,bank_name_more_count,card_type_less_cate,card_type_less_count,card_type_more_cate,card_type_more_count,havaCreditCard,id,phoneSame,phone_less_cate,phone_less_count,phone_more_cate,phone_more_count,phone_num,target,phoneTop3,phone_more_cate_top3,auth_bank_phoneTop3_same
0,a,1,a,1,1.0,1,1.0,1,0,501951980776722440,0,132****2008,1,132****2008,1,1,0.0,132,132,0
1,c,1,c,1,1.0,1,1.0,1,0,525890212484616200,0,187****2500,1,187****2500,1,1,0.0,187,187,0
2,l,1,h,1,1.0,2,2.0,4,1,599309364691472392,0,135****3522,6,135****3522,6,6,0.0,135,135,0
3,m,1,m,1,1.0,1,1.0,1,0,582559913071350024,0,136****8651,1,136****8651,1,1,0.0,136,136,0
4,e,1,e,1,1.0,1,1.0,1,0,599017959926534408,0,137****4241,1,137****4241,1,1,0.0,137,137,0
5,b,1,a,4,1.0,2,2.0,4,1,575388917721731080,0,150****1035,6,150****1035,6,6,0.0,150,150,0
6,c,1,c,1,1.0,1,1.0,1,0,541887664329068552,0,134****2594,1,134****2594,1,1,0.0,134,134,0
7,c,1,c,1,1.0,1,1.0,1,0,605760099452981512,0,158****5755,1,158****5755,1,1,0.0,158,158,0
8,a,1,a,1,1.0,1,1.0,1,0,472286860756914440,0,188****8517,1,188****8517,1,1,0.0,152,188,1
9,i,1,o,1,1.0,3,1.0,3,0,600450091387457800,0,183****2222,3,183****2222,3,3,0.0,183,183,0


In [18]:
fulldata

,bank_name_less_cate,bank_name_less_count,bank_name_more_cate,bank_name_more_count,card_type_less_cate,card_type_less_count,card_type_more_cate,card_type_more_count,havaCreditCard,id,phoneSame,phone_less_cate,phone_less_count,phone_more_cate,phone_more_count,phone_num,target,phoneTop3,phone_more_cate_top3,auth_bank_phoneTop3_same
0,a,1,a,1,1.0,1,1.0,1,0,501951980776722440,0,132****2008,1,132****2008,1,1,0.0,132,132,0
1,c,1,c,1,1.0,1,1.0,1,0,525890212484616200,0,187****2500,1,187****2500,1,1,0.0,187,187,0
2,l,1,h,1,1.0,2,2.0,4,1,599309364691472392,0,135****3522,6,135****3522,6,6,0.0,135,135,0
3,m,1,m,1,1.0,1,1.0,1,0,582559913071350024,0,136****8651,1,136****8651,1,1,0.0,136,136,0
4,e,1,e,1,1.0,1,1.0,1,0,599017959926534408,0,137****4241,1,137****4241,1,1,0.0,137,137,0
5,b,1,a,4,1.0,2,2.0,4,1,575388917721731080,0,150****1035,6,150****1035,6,6,0.0,150,150,0
6,c,1,c,1,1.0,1,1.0,1,0,541887664329068552,0,134****2594,1,134****2594,1,1,0.0,134,134,0
7,c,1,c,1,1.0,1,1.0,1,0,605760099452981512,0,158****5755,1,158****5755,1,1,0.0,158,158,0
8,a,1,a,1,1.0,1,1.0,1,0,472286860756914440,0,188****8517,1,188****8517,1,1,0.0,152,188,1
9,i,1,o,1,1.0,3,1.0,3,0,600450091387457800,0,183****2222,3,183****2222,3,3,0.0,183,183,0


In [21]:
fulldata.to_csv("../input/bank_full_undeal.csv",index = False)
fulldata.columns


Index(['bank_name_less_cate', 'bank_name_less_count', 'bank_name_more_cate',
       'bank_name_more_count', 'card_type_less_cate', 'card_type_less_count',
       'card_type_more_cate', 'card_type_more_count', 'havaCreditCard', 'id',
       'phoneSame', 'phone_less_cate', 'phone_less_count', 'phone_more_cate',
       'phone_more_count', 'phone_num', 'target', 'phoneTop3',
       'phone_more_cate_top3', 'auth_bank_phoneTop3_same'],
      dtype='object')

In [24]:
del fulldata["bank_name_less_cate"]
del fulldata["bank_name_less_count"]
del fulldata["card_type_less_cate"]
del fulldata["card_type_less_count"]
del fulldata["phone_less_cate"]
del fulldata["phone_less_count"]
del fulldata["phone_more_cate"]
print(fulldata.columns)
fulldata.head()

Index(['bank_name_more_cate', 'bank_name_more_count', 'card_type_more_cate',
       'card_type_more_count', 'havaCreditCard', 'id', 'phoneSame',
       'phone_more_count', 'phone_num', 'target', 'phoneTop3',
       'phone_more_cate_top3', 'auth_bank_phoneTop3_same'],
      dtype='object')


,bank_name_more_cate,bank_name_more_count,card_type_more_cate,card_type_more_count,havaCreditCard,id,phoneSame,phone_more_count,phone_num,target,phoneTop3,phone_more_cate_top3,auth_bank_phoneTop3_same
0,a,1,1.0,1,0,501951980776722440,0,1,1,0.0,132,132,0
1,c,1,1.0,1,0,525890212484616200,0,1,1,0.0,187,187,0
2,h,1,2.0,4,1,599309364691472392,0,6,6,0.0,135,135,0
3,m,1,1.0,1,0,582559913071350024,0,1,1,0.0,136,136,0
4,e,1,1.0,1,0,599017959926534408,0,1,1,0.0,137,137,0


In [25]:
del fulldata["phoneTop3"]
fulldata.columns


Index(['bank_name_more_cate', 'bank_name_more_count', 'card_type_more_cate',
       'card_type_more_count', 'havaCreditCard', 'id', 'phoneSame',
       'phone_more_count', 'phone_num', 'target', 'phone_more_cate_top3',
       'auth_bank_phoneTop3_same'],
      dtype='object')

In [28]:
# (['bank_name_less_cate', 'bank_name_less_count', 'bank_name_more_cate',
#        'bank_name_more_count', 'card_type_less_cate', 'card_type_less_count',
#        'card_type_more_cate', 'card_type_more_count', 'havaCreditCard', 'id',
#        'phoneSame', 'phone_less_cate', 'phone_less_count', 'phone_more_cate',
#        'phone_more_count', 'phone_num', 'target'],
#最少的都不要了      （最少的也许是异常值）
from sklearn import preprocessing  
cate_feature = [ 'bank_name_more_cate', 'card_type_more_cate', 'havaCreditCard', 
       'phoneSame',  'phone_more_cate_top3', 'auth_bank_phoneTop3_same' ]
num_feature = ['bank_name_more_count', 'card_type_more_count','phone_more_count',  
        'phone_num']
min_max_scaler = preprocessing.StandardScaler()

for i in range(len(cate_feature)):
    tmp = pd.get_dummies(fulldata[cate_feature[i]],prefix = cate_feature[i])
    fulldata = pd.concat([fulldata,tmp],axis = 1) 
    del fulldata[cate_feature[i]]
num = pd.DataFrame(min_max_scaler.fit_transform(fulldata[num_feature]),columns = num_feature)
fulldata.drop(columns = num_feature,inplace = True)
fulldata = pd.concat([fulldata,num],axis = 1) 
m = fulldata.target.copy()
del fulldata["target"]
fulldata.insert(1,"target",m)
fulldata

,id,target,bank_name_more_cate_a,bank_name_more_cate_b,bank_name_more_cate_c,bank_name_more_cate_d,bank_name_more_cate_e,bank_name_more_cate_f,bank_name_more_cate_g,bank_name_more_cate_h,...,phone_more_cate_top3_186,phone_more_cate_top3_187,phone_more_cate_top3_188,phone_more_cate_top3_189,auth_bank_phoneTop3_same_0,auth_bank_phoneTop3_same_1,bank_name_more_count,card_type_more_count,phone_more_count,phone_num
0,501951980776722440,0.0,1,0,0,0,0,0,0,0,...,0,0,0,0,1,0,-0.519122,-0.582044,-0.604938,-0.613536
1,525890212484616200,0.0,0,0,1,0,0,0,0,0,...,0,1,0,0,1,0,-0.519122,-0.582044,-0.604938,-0.613536
2,599309364691472392,0.0,0,0,0,0,0,0,0,1,...,0,0,0,0,1,0,-0.519122,1.307612,1.680578,1.500924
3,582559913071350024,0.0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,-0.519122,-0.582044,-0.604938,-0.613536
4,599017959926534408,0.0,0,0,0,0,1,0,0,0,...,0,0,0,0,1,0,-0.519122,-0.582044,-0.604938,-0.613536
5,575388917721731080,0.0,1,0,0,0,0,0,0,0,...,0,0,0,0,1,0,3.277598,1.307612,1.680578,1.500924
6,541887664329068552,0.0,0,0,1,0,0,0,0,0,...,0,0,0,0,1,0,-0.519122,-0.582044,-0.604938,-0.613536
7,605760099452981512,0.0,0,0,1,0,0,0,0,0,...,0,0,0,0,1,0,-0.519122,-0.582044,-0.604938,-0.613536
8,472286860756914440,0.0,1,0,0,0,0,0,0,0,...,0,0,1,0,0,1,-0.519122,-0.582044,-0.604938,-0.613536
9,600450091387457800,0.0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,-0.519122,0.677726,0.309269,0.232248


In [33]:
# cc = fulldata[ff]
# cc.insert(0,"id",fulldata.id)
# cc.insert(1,"target",fulldata.target)
# train = fulldata[0:train.shape[0]]
# test = cc[train.shape[0]:]
# del test["target"]
# train.head(),test.head(),train.shape,test.shape
# train.to_csv("../dealinput/train1ok.csv",index = False)
# test.to_csv("../dealinput/test1ok.csv",index = False)
#因为前面merge了一次，所以需要再次merge
train = pd.read_csv("../dealinput/train_bankcard.csv")
test = pd.read_csv("../dealinput/test_bankcard.csv")
train.head(),train.shape,test.shape
kk = pd.DataFrame(train.id)
zz = pd.DataFrame(test.id)
kk,zz
train = fulldata.merge(kk,how = 'inner', on = 'id')
test = fulldata.merge(zz,how = 'inner', on = 'id')
print(train.shape,test.shape)
print(train.isnull().sum(),test.isnull().sum())
train.head(),test.head()

(120929, 67) (47031, 67)
id                            0
target                        0
bank_name_more_cate_a         0
bank_name_more_cate_b         0
bank_name_more_cate_c         0
bank_name_more_cate_d         0
bank_name_more_cate_e         0
bank_name_more_cate_f         0
bank_name_more_cate_g         0
bank_name_more_cate_h         0
bank_name_more_cate_i         0
bank_name_more_cate_j         0
bank_name_more_cate_k         0
bank_name_more_cate_l         0
bank_name_more_cate_m         0
bank_name_more_cate_n         0
bank_name_more_cate_nk        0
bank_name_more_cate_o         0
card_type_more_cate_1.0       0
card_type_more_cate_2.0       0
havaCreditCard_0              0
havaCreditCard_1              0
phoneSame_0                   0
phoneSame_1                   0
phone_more_cate_top3_130      0
phone_more_cate_top3_131      0
phone_more_cate_top3_132      0
phone_more_cate_top3_133      0
phone_more_cate_top3_134      0
phone_more_cate_top3_135      0
               

(                   id  target  bank_name_more_cate_a  bank_name_more_cate_b  \
 0  501951980776722440     0.0                      1                      0   
 1  525890212484616200     0.0                      0                      0   
 2  599309364691472392     0.0                      0                      0   
 3  582559913071350024     0.0                      0                      0   
 4  599017959926534408     0.0                      0                      0   
 
    bank_name_more_cate_c  bank_name_more_cate_d  bank_name_more_cate_e  \
 0                      0                      0                      0   
 1                      1                      0                      0   
 2                      0                      0                      0   
 3                      0                      0                      0   
 4                      0                      0                      1   
 
    bank_name_more_cate_f  bank_name_more_cate_g  bank_name_more_c

In [35]:
del test["target"]
train.to_csv("../dealinput/train_bankcard.csv",index = False)
test.to_csv("../dealinput/test_bankcard.csv",index = False)

In [54]:
#下面开始合并数据集
train11 = pd.read_csv("../dealinput/train1ok.csv")
test11 = pd.read_csv("../dealinput/test1ok.csv")    
train11.head(),test11.head(),train11.shape,test11.shape

(                   id  target  qq_bound_已绑定  qq_bound_未绑定  sex_保密  sex_女  \
 0  501951980776722440     0.0             0             1       0      1   
 1  525890212484616200     0.0             0             1       0      0   
 2  599309364691472392     0.0             1             0       1      0   
 3  582559913071350024     0.0             1             0       1      0   
 4  599017959926534408     0.0             0             1       1      0   
 
    sex_男  wechat_bound_已绑定  wechat_bound_未绑定  auth_time_day_1    ...     \
 0      0                 0                 1                0    ...      
 1      1                 1                 0                0    ...      
 2      0                 1                 0                0    ...      
 3      0                 1                 0                0    ...      
 4      0                 1                 0                0    ...      
 
    phoneTop3_186  phoneTop3_187  phoneTop3_188  Appl_sbm_tm  auth_time_miao  

In [56]:
mm = pd.concat([train.id,train[ff]],axis = 1)
nn =  pd.concat([test.id,test[ff]],axis = 1)
train11 = train11.merge(mm,how = "left",on = "id")
test11 = test11.merge(nn,how = "left",on = "id")
print(train11.isnull().sum(),test11.isnull().sum())
train11.head(),test11.head(),train11.shape,test11.shape

id                            0
target                        0
qq_bound_已绑定                  0
qq_bound_未绑定                  0
sex_保密                        0
sex_女                         0
sex_男                         0
wechat_bound_已绑定              0
wechat_bound_未绑定              0
auth_time_day_1               0
auth_time_day_4               0
auth_time_day_6               0
auth_time_day_8               0
auth_time_day_9               0
auth_time_day_12              0
auth_time_day_13              0
auth_time_day_24              0
auth_time_day_25              0
auth_time_day_26              0
auth_time_day_29              0
auth_time_day_30              0
auth_time_mon_1               0
auth_time_mon_2               0
auth_time_mon_3               0
auth_time_mon_4               0
auth_time_mon_5               0
auth_time_mon_6               0
auth_time_mon_7               0
auth_time_mon_8               0
auth_time_mon_9               0
                             ..
phone_mo

(                   id  target  qq_bound_已绑定  qq_bound_未绑定  sex_保密  sex_女  \
 0  501951980776722440     0.0             0             1       0      1   
 1  525890212484616200     0.0             0             1       0      0   
 2  599309364691472392     0.0             1             0       1      0   
 3  582559913071350024     0.0             1             0       1      0   
 4  599017959926534408     0.0             0             1       1      0   
 
    sex_男  wechat_bound_已绑定  wechat_bound_未绑定  auth_time_day_1    ...      \
 0      0                 0                 1                0    ...       
 1      1                 1                 0                0    ...       
 2      0                 1                 0                0    ...       
 3      0                 1                 0                0    ...       
 4      0                 1                 0                0    ...       
 
    phone_more_cate_top3_185  phone_more_cate_top3_186  \
 0            

In [57]:
# train11.to_csv("../dealinput/train2ok.csv",index = False)
# test11.to_csv("../dealinput/test2ok.csv",index = False)

In [62]:
# train = fulldata[0:train.shape[0]]
# test = fulldata[train.shape[0]:]

# trainx = train.iloc[:,2:]
trainx = train11[ff]
trainy = train11.iloc[:,1]
print("trainx.shape",trainx.shape)
print("trainy.shape",trainy.shape)
from sklearn.cross_validation import train_test_split
x_train,x_valid,y_train,y_valid = train_test_split(trainx,trainy,test_size = 0.2,random_state = 2)  
print("x_train.shape",x_train.shape)
print("x_valid.shape",x_valid.shape)
print("y_train.shape",y_train.shape)
print("y_valid.shape",y_valid.shape)

trainx.shape (120929, 81)
trainy.shape (120929,)
x_train.shape (96743, 81)
x_valid.shape (24186, 81)
y_train.shape (96743,)
y_valid.shape (24186,)


In [63]:
skf = StratifiedKFold(n_splits=5)
xgb1 = XGBClassifier(
#         booster = 'dart',
        learning_rate =0.1,
        n_estimators=5000,
        max_depth=5,
        min_child_weight=5,
        gamma=0.1,
        subsample=0.8,
        colsample_bytree=0.8,
        objective= 'binary:logistic',
        nthread=2,
        scale_pos_weight=1,
        StratifiedKFold = True,          #0.866124	0.008005	0.965337	0.002286
        seed=1)
xgb_param = xgb1.get_xgb_params()
#构建稀疏矩阵，运行更快
xgtrain = xgb.DMatrix(x_train, label = y_train)
mm = xgb.cv(xgb_param, xgtrain, num_boost_round=1000, nfold=3,
    metrics='auc', early_stopping_rounds=50)
mm.iloc[-3:,:]  #
# 0.828775
# 0.002160
# 0.880549
# 0.000719   104 维度  单表

# 70
# 0.600300
# 0.012104
# 0.644789
# 0.007244  bank    删除贡献为0的
# 179
# 0.830636
# 0.002385
# 0.903691
# 0.000808   单表加bank  没有删除贡献为0的列

,test-auc-mean,test-auc-std,train-auc-mean,train-auc-std
188,0.830778,0.003264,0.905339,0.000462
189,0.830804,0.003150,0.905607,0.000408
190,0.830849,0.003164,0.905893,0.000481


In [64]:
importance = 0
for i in range(5):
    xgb1 = XGBClassifier(
    #         booster = 'dart',
            learning_rate =0.1,
            n_estimators=mm.shape[0],
            max_depth=5,
            min_child_weight=5,
            gamma=0.1,
            subsample=0.8,
            colsample_bytree=0.8,
            objective= 'binary:logistic',
            nthread=3,
            scale_pos_weight=1,
            StratifiedKFold = True,          #0.866124	0.008005	0.965337	0.002286
            seed=1)
    xgb1.fit(x_train,y_train)
    prexgb = xgb1.predict_proba(x_valid)[:,1]
    auc = roc_auc_score(y_valid,prexgb)
    print("xgb1 auc",auc)
    importance += xgb1.feature_importances_
importance = importance/5

xgb1 auc 0.8233255585636322
xgb1 auc 0.8233255585636322
xgb1 auc 0.8233255585636322
xgb1 auc 0.8233255585636322
xgb1 auc 0.8233255585636322


In [61]:
print(trainx.columns,importance)
number = 0
ff = []
for i in range(len(importance)):
    if importance[i] >0.001:
        ff.append(trainx.columns[i])
        number += 1
print(number)
ff,len(ff)

Index(['qq_bound_已绑定', 'qq_bound_未绑定', 'sex_保密', 'sex_女', 'sex_男',
       'wechat_bound_已绑定', 'wechat_bound_未绑定', 'auth_time_day_1',
       'auth_time_day_4', 'auth_time_day_6',
       ...
       'phone_more_cate_top3_185', 'phone_more_cate_top3_186',
       'phone_more_cate_top3_187', 'phone_more_cate_top3_188',
       'auth_bank_phoneTop3_same_0', 'auth_bank_phoneTop3_same_1',
       'bank_name_more_count', 'card_type_more_count', 'phone_more_count',
       'phone_num'],
      dtype='object', length=113) [0.02520775 0.00387812 0.00526316 0.00193906 0.01606648 0.01385042
 0.00304709 0.00055402 0.00193906 0.00138504 0.00138504 0.00110803
 0.0099723  0.00110803 0.00277008 0.00110803 0.00138504 0.00083102
 0.00221607 0.00166205 0.00249307 0.00166205 0.00083102 0.00083102
 0.00138504 0.00055402 0.00166205 0.00055402 0.00166205 0.00498615
 0.00166205 0.00304709 0.00110803 0.01551246 0.00304709 0.
 0.00138504 0.00166205 0.00083102 0.00221607 0.00110803 0.00138504
 0.00747922 0.00138504 0.00

(['qq_bound_已绑定',
  'qq_bound_未绑定',
  'sex_保密',
  'sex_女',
  'sex_男',
  'wechat_bound_已绑定',
  'wechat_bound_未绑定',
  'auth_time_day_4',
  'auth_time_day_6',
  'auth_time_day_8',
  'auth_time_day_9',
  'auth_time_day_12',
  'auth_time_day_13',
  'auth_time_day_24',
  'auth_time_day_25',
  'auth_time_day_26',
  'auth_time_day_30',
  'auth_time_mon_1',
  'auth_time_mon_2',
  'auth_time_mon_3',
  'auth_time_mon_6',
  'auth_time_mon_8',
  'auth_time_mon_10',
  'auth_time_mon_12',
  'auth_time_year_2015',
  'auth_time_year_2016',
  'auth_time_year_2017',
  'phoneLastLen_3',
  'phoneLastLen_4',
  'phoneTop3_133',
  'phoneTop3_134',
  'phoneTop3_137',
  'phoneTop3_139',
  'phoneTop3_151',
  'phoneTop3_152',
  'phoneTop3_155',
  'phoneTop3_158',
  'phoneTop3_182',
  'phoneTop3_184',
  'phoneTop3_187',
  'phoneTop3_188',
  'Appl_sbm_tm',
  'auth_time_miao',
  'credit_score',
  'quota',
  'missNumber',
  'overdraft',
  'timeGap',
  'bank_name_more_cate_a',
  'bank_name_more_cate_b',
  'bank_name_m